In [43]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [44]:
df = pd.read_csv("retail_sales_dataset.csv",nrows=1000)


In [45]:
import pandas as pd

# Load dataset
df = pd.read_csv("retail_sales_dataset.csv")  # Update correct file name if different

# Basic cleaning
df_clean = df.copy()
df_clean.dropna(how="all", inplace=True)
df_clean.drop_duplicates(inplace=True)

print("Data Loaded & Cleaned")
df_clean.head()


Data Loaded & Cleaned


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [46]:
# --- 2. Quick info & missing values
print("\nColumns:", df.columns.tolist())
print("\nMissing values per column:")
print(df.isnull().sum())


Columns: ['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age', 'Product Category', 'Quantity', 'Price per Unit', 'Total Amount']

Missing values per column:
Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64


In [47]:
# --- 3. Heuristic column detection (adapt to your column names if needed)
def find_col(df, *names):
    lower = {c.lower(): c for c in df.columns}
    for n in names:
        if n.lower() in lower:
            return lower[n.lower()]
    return None

date_col      = find_col(df_clean, "date", "order_date", "transaction_date", "invoice_date", "purchase_date")
sales_col     = find_col(df_clean, "sales", "sale_amount", "revenue", "total", "total_amount", "amount")
quantity_col  = find_col(df_clean, "quantity", "qty", "units")
product_col   = find_col(df_clean, "product", "product_name", "item", "product_id", "productid")
category_col  = find_col(df_clean, "category", "product_category", "product category", "category_name")
customer_col  = find_col(df_clean, "customer", "customer_id", "cust_id", "buyer_id", "client_id")
price_col     = find_col(df_clean, "price", "unit_price", "unitprice")
discount_col  = find_col(df_clean, "discount", "discount_amount", "discount_rate")

print("\nDetected columns:")
print("Date:", date_col)
print("Sales:", sales_col)
print("Quantity:", quantity_col)
print("Product:", product_col)
print("Category:", category_col)
print("Customer:", customer_col)
print("Price:", price_col)
print("Discount:", discount_col)


Detected columns:
Date: Date
Sales: None
Quantity: Quantity
Product: None
Category: Product Category
Customer: None
Price: None
Discount: None


In [48]:
# --- 4. Convert types
if date_col is not None:
    df_clean[date_col] = pd.to_datetime(df_clean[date_col], errors='coerce')

for c in [sales_col, quantity_col, price_col, discount_col]:
    if c is not None:
        df_clean[c] = pd.to_numeric(df_clean[c], errors='coerce')

In [49]:
# --- 5. Remove duplicates and drop rows missing key fields
before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates().copy()
print(f"Dropped duplicates: {before - df_clean.shape[0]} rows")

key_cols = [c for c in [date_col, sales_col, product_col] if c is not None]
if key_cols:
    df_clean = df_clean.dropna(subset=key_cols).copy()
    print("Shape after dropping rows missing key fields:", df_clean.shape)


Dropped duplicates: 0 rows
Shape after dropping rows missing key fields: (1000, 9)


In [50]:
# --- 6. Attempt to infer quantity if missing (sales / price)
if quantity_col is not None and price_col is not None and sales_col is not None:
    mask = df_clean[quantity_col].isnull() & df_clean[price_col].notnull() & df_clean[sales_col].notnull() & (df_clean[price_col] != 0)
    df_clean.loc[mask, quantity_col] = (df_clean.loc[mask, sales_col] / df_clean.loc[mask, price_col]).round().fillna(1)

# Fill remaining numeric missing values with medians (simple strategy)
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

In [51]:
# --- 7. Descriptive statistics
desc = df_clean.describe(include='all')
desc.to_csv(os.path.join(OUT_DIR, "descriptive_stats.csv"))

In [52]:
# --- 8. KPIs
total_sales = df_clean[sales_col].sum() if sales_col is not None else None
avg_order_value = df_clean[sales_col].mean() if sales_col is not None else None
total_orders = df_clean.shape[0]
unique_customers = df_clean[customer_col].nunique() if customer_col is not None else None
unique_products = df_clean[product_col].nunique() if product_col is not None else None

print("\nKPIs:")
print("Total sales:", total_sales)
print("Average order value:", avg_order_value)
print("Total orders:", total_orders)
print("Unique customers:", unique_customers)
print("Unique products:", unique_products)


KPIs:
Total sales: None
Average order value: None
Total orders: 1000
Unique customers: None
Unique products: None


In [53]:
# --- 9. Time-series analysis (daily & monthly aggregates + plots)
if date_col is not None and sales_col is not None:
    df_clean = df_clean.sort_values(date_col)
    daily = df_clean.set_index(date_col).resample("D").agg({sales_col: "sum"})
    monthly = df_clean.set_index(date_col).resample("M").agg({sales_col: "sum"})
    monthly.to_csv(os.path.join(OUT_DIR, "monthly_sales.csv"))

    plt.figure(figsize=(10,4))
    plt.plot(daily.index, daily[sales_col])
    plt.title("Daily Sales Time Series")
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "daily_sales.png"))
    plt.close()

    plt.figure(figsize=(8,4))
    plt.plot(monthly.index, monthly[sales_col])
    plt.title("Monthly Sales")
    plt.xlabel("Month")
    plt.ylabel("Sales")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "monthly_sales.png"))
    plt.close()

In [54]:
# --- 10. Product & category performance
if product_col is not None and sales_col is not None:
    if quantity_col is not None:
        prod_agg = df_clean.groupby(product_col).agg({sales_col: "sum", quantity_col: "sum"}).sort_values(by=sales_col, ascending=False)
    else:
        prod_agg = df_clean.groupby(product_col).agg({sales_col: "sum"}).sort_values(by=sales_col, ascending=False)
    prod_agg.head(20).to_csv(os.path.join(OUT_DIR, "top_products.csv"))

    plt.figure(figsize=(10,5))
    top10 = prod_agg.head(10)
    plt.bar(top10.index.astype(str), top10[sales_col])
    plt.xticks(rotation=45, ha='right')
    plt.title("Top 10 Products by Sales")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "top_products.png"))
    plt.close()

if category_col is not None and sales_col is not None:
    cat_agg = df_clean.groupby(category_col).agg({sales_col: "sum"}).sort_values(by=sales_col, ascending=False)
    cat_agg.to_csv(os.path.join(OUT_DIR, "category_sales.csv"))
    plt.figure(figsize=(8,4))
    top_cat = cat_agg.head(10)
    plt.bar(top_cat.index.astype(str), top_cat[sales_col])
    plt.xticks(rotation=45, ha='right')
    plt.title("Top Categories by Sales")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "top_categories.png"))
    plt.close()

In [55]:
# --- 11. Customer analysis (top customers by sales)
if customer_col is not None and sales_col is not None:
    if quantity_col is not None:
        cust_agg = df_clean.groupby(customer_col).agg({sales_col: "sum", quantity_col: "sum"}).sort_values(by=sales_col, ascending=False)
    else:
        cust_agg = df_clean.groupby(customer_col).agg({sales_col: "sum"}).sort_values(by=sales_col, ascending=False)
    cust_agg.head(20).to_csv(os.path.join(OUT_DIR, "top_customers.csv"))

    plt.figure(figsize=(10,4))
    top_cust = cust_agg.head(10)
    plt.bar(top_cust.index.astype(str), top_cust[sales_col])
    plt.xticks(rotation=45, ha='right')
    plt.title("Top 10 Customers by Sales")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "top_customers.png"))
    plt.close()

In [56]:
# --- 12. Correlation among numeric features
num_df = df_clean.select_dtypes(include=[np.number])
if not num_df.empty:
    corr = num_df.corr()
    corr.to_csv(os.path.join(OUT_DIR, "correlation_matrix.csv"))
    plt.figure(figsize=(6,6))
    plt.imshow(corr.values, interpolation='nearest', aspect='auto')
    plt.colorbar()
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.title("Correlation Matrix")
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "correlation_matrix.png"))
    plt.close()

In [57]:
OUT_DIR = "./outputs"  # folder created in current working directory

import os
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

print("Saving test file...")
with open(os.path.join(OUT_DIR, "test.txt"), "w") as f:
    f.write("Output successfully saved here!")

print("Check output folder on left panel!")


Saving test file...
Check output folder on left panel!


In [58]:
# --- 13. Save cleaned dataset & short report
cleaned_path = os.path.join(OUT_DIR, "retail_sales_dataset_cleaned.csv")
df_clean.to_csv(cleaned_path, index=False)

report = f"""
EDA summary
Rows after cleaning: {df_clean.shape[0]}
Columns: {df_clean.shape[1]}

KPIs:
- Total sales: {total_sales}
- Average order value: {avg_order_value}
- Total orders: {total_orders}
- Unique customers: {unique_customers}
- Unique products: {unique_products}

Files saved in {OUT_DIR}
"""
with open(os.path.join(OUT_DIR, "eda_report_and_powerbi_guidance.md"), "w") as f:
    f.write(report)

print("EDA complete. Outputs saved to:", OUT_DIR)


EDA complete. Outputs saved to: ./outputs
